# **EXERCÍCIOS – Soluções em Energias Renováveis e Sustentáveis**

**Objetivos de Aprendizagem**

• Diferenciar tarefas de regressão e classificação.

• Aplicar ferramentas de análise em Python (pandas, scikit-learn) e no Orange Data Mining.

• Compreender como dados de energia podem ser usados para apoiar decisões reais

## **Parte 2 – Classificação (Smart Grid Stability)**

**01) Imports**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

**02) Carregar dataset**

In [ ]:
df = pd.read_csv("/content/smart_grid_stability_augmented.csv")
print("Dimensões:", df.shape)
display(df.head())

**03) Verificar colunas e tipos**

In [ ]:
print(df.info())

**04) Definir rótulo**

In [ ]:
target_col = "stabf"

In [ ]:
# Codificar rótulo (stable/unstable -> 0/1)
le = LabelEncoder()
df[target_col] = le.fit_transform(df[target_col])

# Separar features e target
X = df.drop(columns=[target_col])
y = df[target_col]

# Escalar features numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**05) Divisão treino/teste**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
   X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

**06) Modelos**

In [ ]:
models = {
   "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
   "Decision Tree": DecisionTreeClassifier(random_state=42),
   "KNN": KNeighborsClassifier(),
   "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42)
}

results = {}

for name, model in models.items():
   model.fit(X_train, y_train)
   y_pred = model.predict(X_test)

   acc = accuracy_score(y_test, y_pred)
   f1 = f1_score(y_test, y_pred)
   cm = confusion_matrix(y_test, y_pred)

   results[name] = {"accuracy": acc, "f1": f1, "cm": cm}

   print(f"\n=== {name} ===")
   print("Accuracy:", acc)
   print("F1:", f1)
   print("Classification Report:\n", classification_report(y_test, y_pred))

   # Matriz de confusão
   sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
   plt.title(f"Matriz de Confusão - {name}")
   plt.xlabel("Predito")
   plt.ylabel("Real")
   plt.show()

**07) Comparar desempenho**

In [ ]:
df_results = pd.DataFrame(results).T
print("\nResumo dos Resultados:")
display(df_results)

**08) GridSearch para Random Forest**

In [ ]:
param_grid = {
   "n_estimators": [100, 200],
   "max_depth": [None, 10, 20],
   "min_samples_leaf": [1, 2, 4]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring="f1", n_jobs=-1)
grid.fit(X_train, y_train)

print("\nMelhores hiperparâmetros RF:", grid.best_params_)
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)
print("F1 com melhor RF:", f1_score(y_test, y_pred))

## **Conclusão Comparativa dos Modelos**

Os quatro algoritmos de classificação apresentaram desempenhos distintos na tarefa de prever a estabilidade da rede elétrica inteligente:

• **Logistic Regression** obteve acurácia de **99,93%** e F1 de **0,999**, errando apenas oito instâncias. Isso mostra que até mesmo um modelo linear consegue separar bem as classes, embora apresente pequenas falhas em identificar todos os casos de instabilidade.

• **KNN** apresentou desempenho inferior, com acurácia de **95,26%** e F1 de **0,96**. Esse modelo registrou 568 erros (344 falsos positivos e 224 falsos negativos), revelando maior sensibilidade ao ruído e menor capacidade de generalização em comparação com os demais.

• **Decision Tree** alcançou acurácia e F1 perfeitos (**1.0**), classificando todas as amostras corretamente. Apesar disso, por ser um modelo único, está mais sujeito a **overfitting**, pois pode ter memorizado o conjunto de dados em vez de aprender padrões generalizáveis.

• **Random Forest** também atingiu resultados perfeitos (**100% de acurácia e F1**), mas com uma vantagem fundamental: como combina várias árvores de decisão, tende a **generalizar melhor**, sendo mais robusto contra overfitting e variações nos dados.

## **Modelo mais confiável**

Considerando o contexto da aplicação — **detecção de instabilidade em Smart Grids**, onde falsos negativos podem ter consequências graves — o **Random Forest se apresenta como o modelo mais confiável**. Ele alia **desempenho perfeito nos testes** a uma **robustez superior** em relação às demais técnicas, tornando-se a escolha mais indicada para monitoramento e prevenção de falhas em sistemas elétricos inteligentes.